In [1]:
import numpy as np
import torch
import math
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.datasets import make_regression
import datetime

In [2]:
n=400
d=500

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
mu=np.zeros(d)
sigma=np.zeros((d,d))
sigma[0,1]=0.2
sigma[0,2]=0.2
sigma[1,0]=0.2
sigma[1,2]=0.2
sigma[1,3]=0.2
sigma[d-1,d-2]=0.2
sigma[d-1,d-3]=0.2
sigma[d-2,d-1]=0.2
sigma[d-2,d-3]=0.2
sigma[d-2,d-4]=0.2
for i in range (sigma.shape[0]):
     sigma[i,i]=1
for i in range(2,d-2):
    sigma[i,i-2]=0.2
    sigma[i,i-1]=0.2
    sigma[i,i+1]=0.2
    sigma[i,i+2]=0.2
print(sigma)

[[1.  0.2 0.2 ... 0.  0.  0. ]
 [0.2 1.  0.2 ... 0.  0.  0. ]
 [0.2 0.2 1.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 1.  0.2 0.2]
 [0.  0.  0.  ... 0.2 1.  0.2]
 [0.  0.  0.  ... 0.2 0.2 1. ]]


In [5]:
W=np.linalg.inv(sigma)
print(W[0,0])

1.0779310139207985


In [6]:
def SCAD_dev1(t,a,Self_lamda):
    if (t<=Self_lamda ) :
        dev=Self_lamda 
    else: 
        dev=max(a*Self_lamda-t,0)/(a-1)
    return(dev)

In [7]:
def SCAD_dev2 (intheta,X,Y,n,d,lamda):
    dev=torch.zeros(d,1).to(device) 
    for i in range(dev.shape[0]):
        t=abs(intheta[i,:])
        dev[i,:]=SCAD_dev1(t,3,lamda)
        dev[0,0]=0
    return(dev)

In [8]:
def soft(input_theta,X,Y,n,d,sk,dev):
    A2=Y-X.mm(input_theta)
    A1=torch.t(X).mm(A2)
    a=sk/n
    A=input_theta+A1.mul(a)  
    zero1=torch.zeros(d,1).to(device)
    B=torch.abs(A)-dev.mul(sk)
    C=torch.max(B,zero1)
    D=torch.sign(A)
    output=D.mul(C)
    return(output)

In [9]:
def diff(input,X,Y,n,d,sk,dev):
    new=soft(input,X,Y,n,d,sk,dev)
    di=new-input
    epi_new=Y-X.mm(new)
    epi=Y-X.mm(input)
    ##
    Q=torch.t(epi_new).mm(epi_new.div(2*n))
    ##
    A=torch.t(epi).mm(epi.div(2*n))
    B1=-torch.t(X).mm(epi) 
    B=torch.t(di).mm(B1)
    C=torch.t(di).mm(di)/(2*(sk))
    dif=A+B+C-Q
    return(dif)

In [10]:
def choose_s (input,X,Y,n,d,dev):
    k=1
    dd=diff(input,X,Y,n,d,k,dev)
    while dd<0 :
        k=k*(0.1)
        dd=diff(input,X,Y,n,d,k,dev)
    return(k)

In [11]:
def ISTA (input,X,Y,n,d,dev):
    sk=0.075
    output=soft(input,X,Y,n,d,sk,dev)
    while torch.max(torch.abs(input-output))>1e-5:
        sk=choose_s(input,X,Y,n,d,dev)
        input=output
        output=soft(input,X,Y,n,d,sk,dev)
    return(output)



In [12]:
def LLA (X,Y,lamda,n,d):
    intheta=torch.zeros(d,1).to(device)
    dev = SCAD_dev2(intheta,X,Y,n,d,lamda)
    outtheta= ISTA(intheta,X,Y,n,d,dev)
    while torch.max(torch.abs(intheta-outtheta))>1e-5:
        intheta=outtheta
        outtheta=ISTA(intheta,X,Y,n,d,dev)
        dev=SCAD_dev2(intheta,X,Y,n,d,lamda)
    return(outtheta)

In [13]:
def SCAD(X_cuda,Y_cuda,lamda,n,d):
#     start = datetime.datetime.now()
    theta_scad = LLA(X_cuda,Y_cuda,lamda,n,d)
#     end = datetime.datetime.now()
#     print(end-start)
    return(theta_scad)

In [14]:
def cross_validation(X_cuda,Y_cuda,lamda):
    list1=list(range(0, 80))
    list2=list(range(80, 160))
    list3=list(range(160, 240))
    list4=list(range(240, 320))
    list5=list(range(320, 400))
#############################
    indicex1 = torch.LongTensor(list1).to(device)
    indiceX1 = torch.LongTensor(list2+list3+list4+list5).to(device)
    X1=torch.index_select(X_cuda, 0, indiceX1).to(device)
    x1=torch.index_select(X_cuda, 0, indicex1).to(device)
    Y1=torch.index_select(Y_cuda, 0, indiceX1).to(device)
    y1=torch.index_select(Y_cuda, 0, indicex1).to(device)
    t1= SCAD(X1,Y1,lamda,320,500)
    epi1=y1-x1.mm(t1)
    w1=torch.t(epi1).mm(epi1)
###############################    
    indicex2 = torch.LongTensor(list2).to(device)
    indiceX2 = torch.LongTensor(list1+list3+list4+list5).to(device)
    X2=torch.index_select(X_cuda, 0, indiceX2).to(device)
    x2=torch.index_select(X_cuda, 0, indicex2).to(device)
    Y2=torch.index_select(Y_cuda, 0, indiceX2).to(device)
    y2=torch.index_select(Y_cuda, 0, indicex2).to(device)
    t2= SCAD(X2,Y2,lamda,320,500)
    epi2=y2-x2.mm(t2)
    w2=torch.t(epi2).mm(epi2)
############################ 
    indicex3 = torch.LongTensor(list3).to(device)
    indiceX3 = torch.LongTensor(list2+list1+list4+list5).to(device)
    X3=torch.index_select(X_cuda, 0, indiceX3).to(device)
    x3=torch.index_select(X_cuda, 0, indicex3).to(device)
    Y3=torch.index_select(Y_cuda, 0, indiceX3).to(device)
    y3=torch.index_select(Y_cuda, 0, indicex3).to(device)
    t3= SCAD(X3,Y3,lamda,320,500)
    epi3=y3-x3.mm(t3)
    w3=torch.t(epi3).mm(epi3)
#################################
    indicex4 = torch.LongTensor(list4).to(device)
    indiceX4 = torch.LongTensor(list2+list3+list1+list5).to(device)
    X4=torch.index_select(X_cuda, 0, indiceX4).to(device)
    x4=torch.index_select(X_cuda, 0, indicex4).to(device)
    Y4=torch.index_select(Y_cuda, 0, indiceX4).to(device)
    y4=torch.index_select(Y_cuda, 0, indicex4).to(device)
    t4= SCAD(X4,Y4,lamda,320,500)
    epi4=y4-x4.mm(t4)
    w4=torch.t(epi4).mm(epi4)
################################
    indicex5 = torch.LongTensor(list5).to(device)
    indiceX5 = torch.LongTensor(list2+list3+list4+list1).to(device)
    X5=torch.index_select(X_cuda, 0, indiceX5).to(device)
    x5=torch.index_select(X_cuda, 0, indicex5).to(device)
    Y5=torch.index_select(Y_cuda, 0, indiceX5).to(device)
    y5=torch.index_select(Y_cuda, 0, indicex5).to(device)
    t5= SCAD(X5,Y5,lamda,320,500)
    epi5=y5-x5.mm(t5)
    w5=torch.t(epi5).mm(epi5)
##################################
    error=(w1+w2+w3+w4+w5)/400
    print(error)
    return(error)

In [15]:
def sup(beta1):
    b=torch.zeros(500,1).to(device)
    b[beta1!=0]=1
    s=torch.sum(b)
    return(s-1)

In [16]:
sample = np.random.multivariate_normal(mu, W, n)
X_np=sample.T

In [90]:
i=55
X_copy=X_np
Xj_np=np.zeros((1,400))
Xj_np[0,:]=X_copy[i,:]
X_j_np=np.delete(X_copy,[i],axis=0)
X_j_np.shape
#############################
Xj1=Xj_np.astype("float32")
Xj2=torch.from_numpy(Xj1).to(device)
Xj=torch.t(Xj2)
X_j1=X_j_np.astype("float32")
X_j2=torch.from_numpy(X_j1).to(device)
X_j=torch.t(X_j2)
one=torch.ones(400,1).to(device)
X_j_sum=torch.cat((one,X_j),1)
#####################
beta1=SCAD(X_j_sum,Xj,0.11,400,500)
supp=int(sup(beta1))
print(supp)
#########################
XX=X_j_sum.cpu().detach().numpy()
YY=Xj.cpu().detach().numpy()
Y=np.squeeze(YY)
##############################
reg = OrthogonalMatchingPursuit(supp).fit(XX, Y)
p=reg.coef_
###############################
Z=Y-XX.dot(p)
r=Z.T.dot(Z)/(n-supp)
print(r)

0:00:08.598302
11
0.6774772487670018


In [91]:
beta1=SCAD(X_j_sum,Xj,0.08,400,500)

0:00:15.593819


In [92]:
for s in range(0,6):
    cross_validation(X_j_sum,Xj,0.08+0.01*s)

0:00:08.275410
0:00:08.346538
0:00:07.416819
0:00:12.235783
0:00:05.459773
tensor([[0.8296]], device='cuda:0')
0:00:16.802422
0:00:07.758412
0:00:12.983783
0:00:06.920286
0:00:14.444017
tensor([[0.8203]], device='cuda:0')
0:00:09.363555
0:00:12.407128
0:00:08.671890
0:00:09.082471
0:00:12.526433
tensor([[0.8164]], device='cuda:0')
0:00:07.430913
0:00:10.746168
0:00:03.101345
0:00:07.893238
0:00:09.955327
tensor([[0.8148]], device='cuda:0')
0:00:04.793332
0:00:04.928004
0:00:03.884821
0:00:04.627703
0:00:04.742211
tensor([[0.8166]], device='cuda:0')
0:00:04.682214
0:00:04.762705
0:00:03.983932
0:00:03.978144
0:00:04.610135
tensor([[0.8211]], device='cuda:0')


In [60]:

w=p/r
w[0]=1/r
read=np.zeros((500,1))
for k in range(0,i):
    read[k]=w[k+1]
read[i]=w[0]
for k in range(i+1,500):
    read[k]=w[k]

[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.14521931]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.25610759]
 [ 1.11400813]
 [ 0.        ]
 [ 0.29963901]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.     

In [17]:
omega=np.zeros((500,500))

In [20]:
for i in range(0,500):
    start = datetime.datetime.now()
    X_copy=X_np
    Xj_np=np.zeros((1,400))
    Xj_np[0,:]=X_copy[i,:]
    X_j_np=np.delete(X_copy,[i],axis=0)
    X_j_np.shape
    #############################
    Xj1=Xj_np.astype("float32")
    Xj2=torch.from_numpy(Xj1).to(device)
    Xj=torch.t(Xj2)
    X_j1=X_j_np.astype("float32")
    X_j2=torch.from_numpy(X_j1).to(device)
    X_j=torch.t(X_j2)
    one=torch.ones(400,1).to(device)
    X_j_sum=torch.cat((one,X_j),1)
    #####################
    beta1=SCAD(X_j_sum,Xj,0.11,400,500)
    supp=int(sup(beta1))+1
    #########################
    XX=X_j_sum.cpu().detach().numpy()
    YY=Xj.cpu().detach().numpy()
    Y=np.squeeze(YY)
    ##############################
    reg = OrthogonalMatchingPursuit(supp).fit(XX, Y)
    p=reg.coef_
    ###############################
    Z=Y-XX.dot(p)
    r=Z.T.dot(Z)/(n-supp)
    beta=beta1.cpu().detach().numpy()
    w=beta/r
    w[0]=1/r
    read=np.zeros((500,1))
    for k in range(0,i):
        read[k]=w[k+1]
    read[i]=w[0]
    for k in range(i+1,500):
        read[k]=w[k]
    omega[:,i]=read[:,0]
    end = datetime.datetime.now()
    print("loop",i,end-start)

loop 0 0:00:09.787189
loop 1 0:00:06.810390
loop 2 0:00:12.547409
loop 3 0:00:07.448789
loop 4 0:00:11.167550
loop 5 0:00:06.044198
loop 6 0:00:07.183950
loop 7 0:00:06.539008
loop 8 0:00:07.059090
loop 9 0:00:09.128021
loop 10 0:00:06.333153
loop 11 0:00:14.962745
loop 12 0:00:15.825703
loop 13 0:00:15.299428
loop 14 0:00:07.109426
loop 15 0:00:15.528561
loop 16 0:00:14.641215
loop 17 0:00:20.163246
loop 18 0:00:10.317571
loop 19 0:00:13.374308
loop 20 0:00:06.275580
loop 21 0:00:10.428648
loop 22 0:00:09.839278
loop 23 0:00:11.066331
loop 24 0:00:06.762952
loop 25 0:00:09.729359
loop 26 0:00:11.452223
loop 27 0:00:06.597580
loop 28 0:00:13.180830
loop 29 0:00:13.506180
loop 30 0:00:06.879544
loop 31 0:00:10.464332
loop 32 0:00:12.386370
loop 33 0:00:10.185355
loop 34 0:00:05.097648
loop 35 0:00:06.522935
loop 36 0:00:06.392925
loop 37 0:00:07.404895
loop 38 0:00:10.488762
loop 39 0:00:06.933456
loop 40 0:00:12.101444
loop 41 0:00:06.780287
loop 42 0:00:08.474389
loop 43 0:00:06.42044

In [21]:
np.save("math287model1new",omega)

In [58]:
rt=np.load("math287model1new.npy")

In [59]:
ee=rt
for i in range(0,500):
    for j in range(i,500):
        if (np.abs(ee[i,j])<=np.abs(ee[j,i])):
            ee[j,i]=ee[i,j]
        else:
            ee[i,j]=ee[j,i]
  

In [60]:
dd=ee-sigma

In [43]:
print("F:",np.linalg.norm(dd,ord = 'fro'),"Operator；",np.linalg.norm(dd,ord = 2),"1,inf:",np.linalg.norm(dd,ord = np.inf))

F: 9.504053712383124 Operator； 0.8364146453163639 1,inf: 1.6029051155317575


In [61]:
def TP(KK):
    summ=0
    for i in range(2,498):
        for j in range(i-2,i+3):
            if(KK[i,j]!=0):
                summ=summ+1
                KK[i,j]=0
    u=(summ+14)/(5*500-6)
    return(u)
    

In [62]:
TP(ee)

0.9390537289494787

In [65]:
def TN(KK):
    for i in range(2,498):
        for j in range(i-2,i+3):
                KK[i,j]=0
    summ=0             
    for i in range(0,500):
        for j in range(0,500):
            if(KK[i,j]!=0):
                summ=summ+1
    u=(summ-14)/(500*500-2494)
    return(1-u)

In [66]:
TN(ee)

0.9754591807875365